In [1]:
#from bs4 import BeautifulSoup
import requests, json
import pandas as pd

ModuleNotFoundError: No module named 'requests'

In [2]:
# 保存session
reqs = requests.session()

In [3]:
# 登录网站
login_url = 'http://dict.youdao.com/login/acc/login?cf=3'
post_data = {'username': 'coolzhuang@163.com',
'password': '4dc16e7394b7bfd3625df3da6f21db15',
'app': 'web',
'product': 'DICT',
'tp': 'urstoken',
'cf': '3',
'fr': '1',
'ru': 'http://c.youdao.com/dict/commons/oncallback.html',
'er': 'http://c.youdao.com/dict/commons/oncallback.html'}
reqs.post(login_url, post_data)
print(reqs.cookies.get_dict())

{'DICT_LOGIN': '1||1600853953778', 'DICT_SESS': 'v2|Bg7r2ZNaW0eBOMzY0fOGRlEn46Z0MzfRTuPLgZ0Lz50OYhHg4nHpZ0wy0fqz6MzW0J4P4kfO4gBRqBRHkG6Mlm0TyP4YGhfPB0'}


In [4]:
# 查看任务列表
url_task = 'http://zb.youdao.com/crowd/task/page/list?currentPage=1&pageSize=50'
html_task = reqs.get(url_task)
str_json = html_task.content.decode(encoding='utf8')
list_task = json.loads(str_json)
df_task = pd.DataFrame(list_task['data']['content'])

In [5]:
# 更新任务细节
l_d_task_item = []
for task in list_task['data']['content']:
    # print(task)
    url_task_item = 'http://zb.youdao.com/crowd/task/homepage/detail?id={}'.format(task['id'])
    list_task_item = json.loads(reqs.get(url_task_item).content.decode('utf8'))
    dict_task_item = list_task_item['data']
    dict_task_item['id'] = task['id']
    l_d_task_item.append(dict_task_item)

df_task_item_detail = pd.DataFrame(l_d_task_item)

In [6]:
# 更新URL字段
df_task_item_detail['url'] = df_task_item_detail.id.apply(lambda x : 'http://zb.youdao.com/taskDescription.html?id={}'.format(x))

In [7]:
# 连接数据并显示可用单价较高的项目
df_task_detail = df_task.merge(df_task_item_detail, how='outer', on='id')

df_available_task = df_task_detail[['id','jobTitle_x','taskNum_y','price_x','skipPrice','allowClaimTask','allowClaimCheckTask','url']][df_task_detail.taskNum_y > 0].sort_values('skipPrice', ascending= False)

In [8]:
df_available_task

,id,jobTitle_x,taskNum_y,price_x,skipPrice,allowClaimTask,allowClaimCheckTask,url
2,1388,【新手可领】英文长音频转写 2020.9.23,299,3.01,3.010,1,2.0,http://zb.youdao.com/taskDescription.html?id=1388
4,1386,【新手可领】图片内容转写 2020.9.21,1,2.00,0.050,1,2.0,http://zb.youdao.com/taskDescription.html?id=1386
6,1378,【新手可领】中文短音频转写（需调时间）2020.9.14,319,1.32,0.033,1,2.0,http://zb.youdao.com/taskDescription.html?id=1378
5,1382,英式音标编辑（2）- 2020.9.17,414,7.50,0.030,1,2.0,http://zb.youdao.com/taskDescription.html?id=1382
1,1384,【新手可领】中文短音频转写（需加标点）2020.9.18,612,1.05,0.021,1,2.0,http://zb.youdao.com/taskDescription.html?id=1384


In [9]:
# 查看任务结果
url_task_rlt = 'http://zb.youdao.com/crowd/task/person/list?state=-1&currentPage=1&pageSize=50'
list_task_rlt = json.loads(reqs.get(url_task_rlt).content.decode('utf8'))
df_task_rlt = pd.DataFrame(list_task_rlt['data']['content'])
df_task_rlt[['taskName','projectId','taskId','status','accuracy']]

,taskName,projectId,taskId,status,accuracy
0,【新手可领】中学生字迹英文图片转写2020/7/14,1255,863960,PAID,1.000
1,【新手可领】中学生字迹英文图片转写2020/7/14,1255,863943,PAID,1.000
2,【新手可领】图片内容转写 2020.7.2,1223,844283,PAID,0.925
3,【新手可领】图片内容转写 2020.7.2,1223,844093,PAID,0.975
4,【新手可领】图片内容转写 2020.6.29,1215,840863,PAID,0.975
5,【新手可领】图片内容转写 2020.6.29,1215,840843,UNQUALIFIED,0.600
6,【新手可领】中学生字迹英文图片转写2020/6/28,1205,838847,PAID,1.000
7,【新手可领】中学生字迹英文图片转写2020/6/28,1205,838819,PAID,1.000
8,【新手可领】图片内容转写 2020.6.29,1215,840539,UNQUALIFIED,0.900
9,【新手可领】中学生字迹英文图片转写2020/6/28,1205,838523,PAID,1.000


In [10]:
# 酬劳信息
url_wage_rlt = 'http://zb.youdao.com/crowd/task/account/list?currentPage=1&pageSize=500'
list_wage_rlt = json.loads(reqs.get(url_wage_rlt).content.decode('utf8'))
df_wage_rlt = pd.DataFrame(list_wage_rlt['data']['content'])
df_wage_rlt

,checkTime,taskName,type,projectId,taskId,wage,status
0,1595497208000,【新手可领】中学生字迹英文图片转写2020/7/14,0,1255,863960,1.84,PAID
1,1595498516000,【新手可领】中学生字迹英文图片转写2020/7/14,0,1255,863943,1.84,PAID
2,1594368900000,【新手可领】图片内容转写 2020.7.2,0,1223,844283,1.11,PAID
3,1594279907000,【新手可领】图片内容转写 2020.7.2,0,1223,844093,1.17,PAID
4,1593511892000,【新手可领】图片内容转写 2020.6.29,0,1215,840863,1.17,PAID
5,1594563122000,【新手可领】中学生字迹英文图片转写2020/6/28,0,1205,838847,1.84,PAID
6,1593758475000,【新手可领】中学生字迹英文图片转写2020/6/28,0,1205,838819,1.84,PAID
7,1594615268000,【新手可领】中学生字迹英文图片转写2020/6/28,0,1205,838523,1.84,PAID
8,1594610795000,【新手可领】中学生字迹英文图片转写2020/6/28,0,1205,838385,1.84,PAID
9,1593571865000,【新手可领】中学生字迹英文图片转写2020/6/28,0,1205,837939,1.84,PAID


In [11]:
# df_task_detail[['id','jobTitle_x','taskNum_y','price_x','skipPrice','allowClaimTask','allowClaimCheckTask','url']]
pd.merge(df_task_rlt, df_wage_rlt, how = 'outer', 
         left_on=['taskId','projectId','taskName','type','status'], 
         right_on=['taskId','projectId','taskName','type','status'])[['projectId','taskId','taskName','accuracy','status','wage']]

,projectId,taskId,taskName,accuracy,status,wage
0,1255,863960,【新手可领】中学生字迹英文图片转写2020/7/14,1.000,PAID,1.84
1,1255,863943,【新手可领】中学生字迹英文图片转写2020/7/14,1.000,PAID,1.84
2,1223,844283,【新手可领】图片内容转写 2020.7.2,0.925,PAID,1.11
3,1223,844093,【新手可领】图片内容转写 2020.7.2,0.975,PAID,1.17
4,1215,840863,【新手可领】图片内容转写 2020.6.29,0.975,PAID,1.17
5,1215,840843,【新手可领】图片内容转写 2020.6.29,0.600,UNQUALIFIED,NaN
6,1205,838847,【新手可领】中学生字迹英文图片转写2020/6/28,1.000,PAID,1.84
7,1205,838819,【新手可领】中学生字迹英文图片转写2020/6/28,1.000,PAID,1.84
8,1215,840539,【新手可领】图片内容转写 2020.6.29,0.900,UNQUALIFIED,NaN
9,1205,838523,【新手可领】中学生字迹英文图片转写2020/6/28,1.000,PAID,1.84


In [12]:
df_wage_rlt.wage.sum()

18.17